# Autoencoders

## Seoul AI Meetup, July 8

Martin Kersner, <m.kersner@gmail.com>

Structure of this presentation is largely based on chapter *15: Autoencoders* from book [Hands-On Machine Learning with Scikit-Learn & Tensorflow](http://shop.oreilly.com/product/0636920052289.do).

### Content
1. Efficient Data Representation
1. Principal Component Analysis (PCA)
1. t-SNE
1. Stacked Autoencoders
1. Unsupervised Pretraining Using Stacked Autoencoders
1. Denoising Autoencoders
1. Sparse Autoencoders
1. Variational Autoencoders
1. Other Autoencoders

## Efficient Data Representation

* Number sequences
    * 56, 46, 8, 56, 7, 6, 8, 52,... 
    * 5, 16, 8, 4, 2, 1, 4, 2, 1,...

## Efficient Data Representation

* Lower Data Dimensionality
  * Reduced computational cost
  * Easier to train ([Curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality))
  * Easier to visualize (ND -> 3D or ND -> 2D)
* [Factor Analysis](https://en.wikipedia.org/wiki/Factor_analysis), [Independent Component Analysis](https://en.wikipedia.org/wiki/Independent_component_analysis)

## Principal Component Analysis

* For unlabeled data
* Transformation from original coordinate system to the new one
* Orthogonal linear transformation
* Used for dimensionality reduction
* Principal components represent directions along which the data has the largest variations
* [sklearn.decomposition.PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

In [1]:
# scikit-learn: Principal Component Analysis
import numpy as np
from sklearn.decomposition import PCA

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
pca.fit(X)

print(pca.explained_variance_ratio_)

[ 0.99244289  0.00755711]


## Autoencoders

* Artificial Neural Networks
* Trained to efficiently encode (**codings**) input information
* Purposes
  * Decrease dimensionality
  * Feature detectors (unsupervised pretraining for deep neural networks)
  * Randomly generate new data

* Encoder (recognition network)
* Decoder (generative network)

## Variational Autoencoders